In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
#Checking if pytorch is installed
torch.__version__

'1.7.1+cpu'

In [3]:
#importing
movies = pd.read_csv('ml-1m/movies.dat',sep='::',header=None,engine='python',encoding='latin-1')
movies.head(10)
users = pd.read_csv('ml-1m/users.dat',sep='::',header=None,engine='python',encoding='latin-1')
users.head(10) #2nd column - gender , 3rd column - age, 4th column - occupation 5th column - zip code
ratings = pd.read_csv('ml-1m/ratings.dat',sep='::',header=None,engine='python',encoding='latin-1')
ratings.head(10) #1st column users 2nd column - movies(movie id) 3rd column -  ratings 4th column - dont care lol. they are timestamps

,0,1,2,3
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
5,1,1197,3,978302268
6,1,1287,5,978302039
7,1,2804,5,978300719
8,1,594,4,978302268
9,1,919,4,978301368


In [4]:
#preparing test and train set
training_set = pd.read_csv('ml-100k/u1.base',delimiter='\t',header=None)
training_set.head(10)

,0,1,2,3
0,1,1,5,874965758
1,1,2,3,876893171
2,1,3,4,878542960
3,1,4,3,876893119
4,1,5,3,889751712
5,1,7,4,875071561
6,1,8,1,875072484
7,1,9,5,878543541
8,1,11,2,875072262
9,1,13,5,875071805


In [5]:
#converting training set to array
training_set = np.array(training_set, dtype='int')
training_set

array([[        1,         1,         5, 874965758],
       [        1,         2,         3, 876893171],
       [        1,         3,         4, 878542960],
       ...,
       [      943,      1188,         3, 888640250],
       [      943,      1228,         3, 888640275],
       [      943,      1330,         3, 888692465]])

In [6]:
test_set = pd.read_csv('ml-100k/u1.test',delimiter='\t',header=None)
test_set = np.array(test_set, dtype='int')
test_set

array([[        1,         6,         5, 887431973],
       [        1,        10,         3, 875693118],
       [        1,        12,         5, 878542960],
       ...,
       [      459,       934,         3, 879563639],
       [      460,        10,         3, 882912371],
       [      462,       682,         5, 886365231]])

In [7]:
#Getting number of users and movies
nb_users = int(max(max(training_set[:,0]),max(test_set[:,0])))
nb_movies = int(max(max(training_set[:,1]),max(test_set[:,1])))
print(nb_users,nb_movies)

943 1682


In [8]:
#Converting data to array with users in lines and movies in columns
def convert(data):
    new_data = []
    for id_users in range(1, nb_users+1):
        id_movies = data[:,1][data[:,0] == id_users] #movies what the user watched
        id_ratings = data[:,2][data[:,0] == id_users] #movies rated by the user
        ratings = np.zeros(nb_movies)
        ratings[id_movies-1] = id_ratings
        new_data.append(list(ratings))
    return new_data

training_set = convert(training_set)
test_set = convert(test_set)

In [9]:
len(training_set)

943

In [10]:
#Converting to torch tensors
#tensors - array contain elements of a single datatype
training_set = torch.FloatTensor(training_set)
test_set = torch.FloatTensor(test_set)

In [11]:
training_set

tensor([[5., 3., 4.,  ..., 0., 0., 0.],
        [4., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [5., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 5., 0.,  ..., 0., 0., 0.]])

In [12]:
#Converting the ratings into binary ratings 1 (Liked) or 0 (non liked)
training_set[training_set == 0] = -1 #movie isn't rated
training_set[training_set == 1] = 0
training_set[training_set == 2] = 0
training_set[training_set >= 3] = 1

test_set[test_set == 0] = -1 #movie isn't rated
test_set[test_set == 1] = 0
test_set[test_set == 2] = 0
test_set[test_set >= 3] = 1

In [ ]:
#Creating our NN
class RBM():
    def __init__(self,nv,nh): #nv - visible nodes nh - hidden nodes
        self.W = torch.randn(nh,nv) #initialises the weights
        self.a = torch.randn(1,nh) #creating a 2D tensor, nh corresponds to the bias
        self.b = torch.randn(1,nv) #creating the bias for the visible nodes
        
        